# 6. Decision Tree and Ensemble Learning
Exercice issu de ML Zoomcamp 6 [https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/06-trees](https://github.com/DataTalksClub/machine-learning-zoomcamp/tree/master/06-trees)
Reprise de son tutoriel avec certaines modifications et commentaires

## 6.1 Credit risk scoring project
* Dataset: https://github.com/gastonstat/CreditScoring

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

## 6.2 Data cleaning and preparation

In [3]:
data_url = 'https://raw.githubusercontent.com/gastonstat/CreditScoring/master/CreditScoring.csv'

In [9]:
import os
filename = 'CreditScoring.csv'
if not os.path.exists(filename):
    os.system(f'wget {data_url} -O {filename}')
    print(f"Le fichier {filename} a été téléchargé avec succès.")
else:
    # Si le fichier existe déjà, imprimez un message
    print(f"Le fichier {filename} existe déjà.")

--2024-01-18 15:32:35--  https://raw.githubusercontent.com/gastonstat/CreditScoring/master/CreditScoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 

Le fichier CreditScoring.csv a été téléchargé avec succès.


200 OK
Length: 182489 (178K) [text/plain]
Saving to: 'CreditScoring.csv'

     0K .......... .......... .......... .......... .......... 28% 1.58M 0s
    50K .......... .......... .......... .......... .......... 56% 3.95M 0s
   100K .......... .......... .......... .......... .......... 84% 6.63M 0s
   150K .......... .......... ........                        100% 2.33M=0.06s

2024-01-18 15:32:36 (2.79 MB/s) - 'CreditScoring.csv' saved [182489/182489]



In [10]:
!head CreditScoring.csv

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
"Status","Seniority","Home","Time","Age","Marital","Records","Job","Expenses","Income","Assets","Debt","Amount","Price"
1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
1,0,1,36,26,1,1,1,46,107,0,0,310,910
1,1,2,60,36,2,1,1,75,214,3500,0,650,1645
1,29,2,60,44,2,1,1,75,125,10000,0,1600,1800
1,9,5,12,27,1,1,1,35,80,0,0,200,1093
1,0,2,60,32,2,1,3,90,107,15000,0,1200,1957


In [36]:
df = pd.read_csv(filename)
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


On remarque que le nom des colonnes du dataset contient des majuscules 

In [37]:
def clean_headers(df:pd.DataFrame)->pd.DataFrame:
    df.columns = df.columns.str.lower().str.replace(' ','_')
    return df

def clean_rows(df:pd.DataFrame)->pd.DataFrame:
    for column in df.columns:
        if pd.api.types.is_string_dtype(df[column]):
            df[column] = df[column].str.lower().str.replace(' ', '_')
    return df

def clean_headers_rows(df:pd.DataFrame)->pd.DataFrame:
    df = clean_headers(df)
    df = clean_rows(df)
    return df

In [38]:
df = clean_headers_rows(df)
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


Il nous manque des informations pour comprendre le dataset, elles sont disponibles sur le github d'où provient de fichier csv. Les voici sous forme de dictionnaire:
status = {0:"unknow", 1:"good", 2:"bad"}
home = {0:"unknow", 1:"rent", 2:"owner", 3:"priv", 4:"ignore", 5:"parents", 6:"other"}
marital = {0:"unknow", 1:"single", 2:"married", 3:"widow", 4:"separated", 5:"divorced"}
records = {0:"unknow", 1:"no_rec", 2:"yes_rec"}
job = {0:"unknow", 1:"fixed", 2:"partime", 3:"freelance", 4:"other"}

On va créer un dataframe lisible pour mieux comprendre les données:

In [39]:
dict_status = {0:"unknow", 1:"good", 2:"bad"}
home = {0:"unknow", 1:"rent", 2:"owner", 3:"priv", 4:"ignore", 5:"parents", 6:"other"}
marital = {0:"unknow", 1:"single", 2:"married", 3:"widow", 4:"separated", 5:"divorced"}
records = {0:"unknow", 1:"no_rec", 2:"yes_rec"}
job = {0:"unknow", 1:"fixed", 2:"partime", 3:"freelance", 4:"other"}
df_readable = df.copy()
df_readable.status = df.status.map(dict_status)
df_readable['home'] = df.status.map(home)
df_readable['marital'] = df.status.map(marital)
df_readable['records'] = df.status.map(records)
df_readable['job'] = df.status.map(job)


In [40]:
df_readable.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,good,9,rent,60,30,single,no_rec,fixed,73,129,0,0,800,846
1,good,17,rent,60,58,single,no_rec,fixed,48,131,0,0,1000,1658
2,bad,10,owner,36,46,married,yes_rec,partime,90,200,3000,0,2000,2985
3,good,0,rent,60,24,single,no_rec,fixed,63,182,2500,0,900,1325
4,good,0,rent,36,26,single,no_rec,fixed,46,107,0,0,310,910


Dans les sources, il est indiqué que les valeurs manquantes sont remplacées par le nombre 99999999. Nous devons donc remplacer par np.nan afin de ne pas fausser les résultats

In [42]:
df.describe().round()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0
mean,1.0,8.0,3.0,46.0,37.0,2.0,1.0,2.0,56.0,763317.0,1060341.0,404382.0,1039.0,1463.0
std,0.0,8.0,2.0,15.0,11.0,1.0,0.0,1.0,20.0,8703625.0,10217569.0,6344253.0,475.0,628.0
min,0.0,0.0,0.0,6.0,18.0,0.0,1.0,0.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,1.0,2.0,2.0,36.0,28.0,2.0,1.0,1.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,1.0,5.0,2.0,48.0,36.0,2.0,1.0,1.0,51.0,120.0,3500.0,0.0,1000.0,1400.0
75%,2.0,12.0,4.0,60.0,45.0,2.0,1.0,3.0,72.0,166.0,6000.0,0.0,1300.0,1692.0
max,2.0,48.0,6.0,72.0,68.0,5.0,2.0,4.0,180.0,99999999.0,99999999.0,99999999.0,5000.0,11140.0


On constate avec la commande ```df.describe().round()``` en valeur **max** nous avons income, assets et debt qui contiennent le nombre 99999999

In [43]:
for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

In [45]:
df.describe().round()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
count,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4455.0,4421.0,4408.0,4437.0,4455.0,4455.0
mean,1.0,8.0,3.0,46.0,37.0,2.0,1.0,2.0,56.0,131.0,5403.0,343.0,1039.0,1463.0
std,0.0,8.0,2.0,15.0,11.0,1.0,0.0,1.0,20.0,86.0,11573.0,1246.0,475.0,628.0
min,0.0,0.0,0.0,6.0,18.0,0.0,1.0,0.0,35.0,0.0,0.0,0.0,100.0,105.0
25%,1.0,2.0,2.0,36.0,28.0,2.0,1.0,1.0,35.0,80.0,0.0,0.0,700.0,1118.0
50%,1.0,5.0,2.0,48.0,36.0,2.0,1.0,1.0,51.0,120.0,3000.0,0.0,1000.0,1400.0
75%,2.0,12.0,4.0,60.0,45.0,2.0,1.0,3.0,72.0,165.0,6000.0,0.0,1300.0,1692.0
max,2.0,48.0,6.0,72.0,68.0,5.0,2.0,4.0,180.0,959.0,300000.0,30000.0,5000.0,11140.0


@7:32 https://www.youtube.com/watch?v=tfuQdI3YO2c&list=PL3MmuxUbc_hIhxl5Ji8t4O6lPAOpHaCLR&index=58